In [12]:
# importing
import json
import csv

import numpy as np
import pandas as pd
from torchvision.io import read_video, write_video
import numpy as np
import os
from joblib import Parallel, delayed



In [13]:
# CONSTANTS

FRAMES_PER_SEC = 30

# UCF_CRIME_DATASET
TRAIN_ANNOTATIONS_SECS_LEVEL = '../data/raw/ucfcrime_v2/public_ucfcrimev2/annotations/train.json'
TEST_ANNOTATIONS_SECS_LEVEL = '../data/raw/ucfcrime_v2/public_ucfcrimev2/annotations/test.json'

# VIDEO LEVEL FILE PATHS
PATH_TRAIN_VIDEO_LEVEL = '../data/interim/train/'
PATH_TEST_VIDEO_LEVEL = '../data/interim/test/'

# VIDEO_PATHS
VIDEO_PATHS = '../data/raw/'

# CHOSEN CRIME ACTION THAT CAN BE OCCURRED IN CAMPUS
CRIME_ACTION = ['Abuse', 'Fighting', 'Vandalism']

In [14]:
with open(TRAIN_ANNOTATIONS_SECS_LEVEL, 'r') as f:
    temporal_annotation = json.load(f)
temporal_annotation_copy = temporal_annotation.copy()
for d in temporal_annotation:
    if all(word not in d for word in CRIME_ACTION):
        del temporal_annotation_copy[d]

In [15]:
temporal_annotation_copy

{'Abuse/Abuse001_x264.mp4': [{'end': 10, 'start': 8},
  {'end': 36, 'start': 33},
  {'end': 61, 'start': 59},
  {'end': 78, 'start': 76}],
 'Abuse/Abuse002_x264.mp4': [{'end': 13, 'start': 3}],
 'Abuse/Abuse003_x264.mp4': [{'end': 42, 'start': 30},
  {'end': 80, 'start': 68},
  {'end': 104, 'start': 94},
  {'end': 115, 'start': 108}],
 'Abuse/Abuse004_x264.mp4': [{'end': 66, 'start': 28},
  {'end': 181, 'start': 107}],
 'Abuse/Abuse005_x264.mp4': [{'end': 4, 'start': 2}, {'end': 30, 'start': 24}],
 'Abuse/Abuse006_x264.mp4': [{'end': 85, 'start': 10},
  {'end': 144, 'start': 100}],
 'Abuse/Abuse007_x264.mp4': [{'end': 5, 'start': 2}],
 'Abuse/Abuse008_x264.mp4': [{'end': 151, 'start': 147}],
 'Abuse/Abuse009_x264.mp4': [{'end': 12, 'start': 8},
  {'end': 20, 'start': 14},
  {'end': 25, 'start': 22}],
 'Abuse/Abuse010_x264.mp4': [{'end': 6, 'start': 2}],
 'Abuse/Abuse011_x264.mp4': [{'end': 17, 'start': 10},
  {'end': 26, 'start': 20},
  {'end': 43, 'start': 31},
  {'end': 94, 'start': 

In [16]:
def video_name(video_path):
    split_path = video_path.split("/")
    video_name_ext = split_path[-1]
    crime_name = split_path[0]
    vid_name = video_name_ext.split(".")[0]
    video_ext = video_name_ext.split(".")[-1]
    return crime_name, vid_name

In [17]:
# for video_pth in temporal_annotation_copy:
#     video_temporal = temporal_annotation_copy[video_pth]
#     video_temporal = pd.DataFrame(video_temporal)
#     video_temporal['period'] = video_temporal['end'] - video_temporal['start']
#     video_temporal = video_temporal[video_temporal['period'] > 2]
#     videos = {}
#
#     for i in range(len(video_temporal)):
#         videos[i] = np.arange(int(video_temporal['start'][i]), int(video_temporal['end'][i]), 3)
#
#     for video in videos:
#         for i in range(len(videos[video])):
#             if i < len(videos[video]) - 1:
#                 start = videos[video][i]
#                 end = videos[video][i + 1]
#                 full_path = VIDEO_PATHS + video_pth
#                 crime_name, vid_name = video_name(video_pth)
#                 write_path = PATH_TRAIN_VIDEO_LEVEL + crime_name
#                 video_write_path = write_path + '/' + vid_name + f'_{start}_{end}' + '.mp4'
#                 os.makedirs(write_path, exist_ok=True)
#                 print(video_write_path)
#                 frames, _, _ = read_video(full_path, start_pts=start, end_pts=end, pts_unit='sec')
#                 write_video(video_write_path, frames, fps=30)


In [18]:
def writing_csv_train(video_write_path, crime_name):
    # Define the actual data
    row_data = [video_write_path, crime_name]

    # 1. Open a new CSV file
    with open(PATH_TRAIN_VIDEO_LEVEL + 'train.csv', 'a') as file:
        # 2. Create a CSV writer
        writer = csv.writer(file)
        # 3. Write data to the file
        writer.writerow(row_data)

In [19]:
def write_training_videos(video_pth):
    video_temporal = temporal_annotation_copy[video_pth]
    video_temporal = pd.DataFrame(video_temporal)
    video_temporal['period'] = video_temporal['end'] - video_temporal['start']
    video_temporal = video_temporal[video_temporal['period'] > 4]
    videos = {}
    for i in range(len(video_temporal)):
        videos[i] = np.arange(int(video_temporal['start'].iloc[i]), int(video_temporal['end'].iloc[i]), 5)
        for video in videos:
            for i in range(len(videos[video])):
                if i < len(videos[video]) - 1:
                    start = videos[video][i]
                    end = videos[video][i + 1]
                    full_path = VIDEO_PATHS + video_pth
                    crime_name, vid_name = video_name(video_pth)
                    write_path = PATH_TRAIN_VIDEO_LEVEL + crime_name
                    video_write_path = write_path + '/' + vid_name + f'_{start}_{end}' + '.mp4'
                    os.makedirs(write_path, exist_ok=True)
                    # print(video_write_path)
                    frames, _, _ = read_video(full_path, start_pts=start, end_pts=end, pts_unit='sec')
                    write_video(video_write_path, frames, fps=30)
                    writing_csv_train(video_write_path, crime_name)

In [20]:
Parallel(n_jobs=os.cpu_count(), prefer='threads', verbose=10)(
    delayed(write_training_videos)(video_pth) for video_pth in temporal_annotation_copy)


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    7.4s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  1.2min
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:  1.9min
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:  3.3min
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:  4.3min
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:  5.6min
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:  6.8min
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:  7.8min
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:  9.0min
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:  9.8min
[Parallel(n_jobs=16)]: Done 159 out of 172 | elapsed: 11.3min remaining:   55.4s
[Parallel(n_jobs=16)]: Done 172 out of 172 | elapsed: 14.0min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [67]:
l = pd.DataFrame(l)
l['period'] = l['end'] - l['start']
l = l[l['period'] > 2]
videos = {}

for i in range(len(l)):
    videos[i] = np.arange(int(l['start'][i]), int(l['end'][i]), 3)


In [69]:
videos

{0: array([30, 33, 36, 39]),
 1: array([68, 71, 74, 77]),
 2: array([ 94,  97, 100, 103]),
 3: array([108, 111, 114])}

In [76]:
for video in videos:
    for i in range(len(videos[video])):
        if i < len(videos[video]) - 1:
            start = videos[video][i]
            end = videos[video][i + 1]
            print(start, end)


30 33
33 36
36 39
68 71
71 74
74 77
94 97
97 100
100 103
108 111
111 114


In [3]:
video_name('Stealing/Stealing101_x264.mp4')

('Stealing', 'Stealing101_x264')

In [ ]:
def create_video_dataset():
    crime_name, vid_name = video_name(video_path)


label_frame_level = PATH_FRAME_LEVEL + video_name + '.csv'
